In [1]:
from utils.imports import *

('/Volumes/solo/ali/Data/train_cls_cube_64/train/false/', u'maked')
('/Volumes/solo/ali/Data/train_cls_cube_64/train/true/', u'maked')


Using TensorFlow backend.


In [2]:
def create_cls_sample(df_anno,df_pred,img_file,data_path,output_true,output_false):
    mini_df_anno = df_anno[df_anno["file"]==img_file] #get all nodules associate with file
    mini_df_pred = df_pred[df_pred["file"]==img_file]
    if mini_df_anno.shape[0]>0: # some files may not have a nodule--skipping those 
        patient_id = img_file[:-9]
        img_array = np.load(data_path + img_file)
        pos_annos = pd.read_csv(data_path + img_file[:-9] + '_annos_pos.csv')
        origin = np.array([pos_annos.loc[0]['origin_x'],pos_annos.loc[0]['origin_y'],pos_annos.loc[0]['origin_z']]) 
        spacing = np.array([pos_annos.loc[0]['spacing_x'],pos_annos.loc[0]['spacing_y'],pos_annos.loc[0]['spacing_z']]) 
        
        for node_idx1, cur_row1 in mini_df_anno.iterrows():       
            node_x = cur_row1["coordX"]
            node_y = cur_row1["coordY"]
            node_z = cur_row1["coordZ"]
            diam = cur_row1["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            
            new_x = int(v_center[0])
            new_y = int(v_center[1])
            new_z = int(v_center[2])
            trainX_1 =  img_array[new_z - 32: new_z + 32,
                                  new_y - 32 : new_y + 32,
                                  new_x - 32 : new_x + 32] 
            
            if trainX_1.shape == (64,64,64):
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_0_')+ '_3d_20_6_i.npy',trainX_1)
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_1_')+ '_3d_20_6_i.npy',np.fliplr(trainX_1))
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_2_')+ '_3d_20_6_i.npy',np.flipud(trainX_1))
            
            new_z1 = new_z + random.choice([-2,-1,1,2])
            new_y1 = new_y + random.choice([-2,-1,1,2])
            new_x1 = new_x + random.choice([-2,-1,1,2])
            
            trainX_2 =  img_array[new_z1 - 32: new_z1 + 32,
                                  new_y1 - 32 : new_y1 + 32,
                                  new_x1 - 32 : new_x1 + 32] 
            
            if trainX_2.shape == (64,64,64):
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_3_')+ '_3d_20_6_i.npy',trainX_2)
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_4_')+ '_3d_20_6_i.npy',np.fliplr(trainX_2))
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_5_')+ '_3d_20_6_i.npy',np.flipud(trainX_2))

                
            new_z2 = new_z + random.choice([-2,-1,1,2])
            new_y2 = new_y + random.choice([-2,-1,1,2])
            new_x2 = new_x + random.choice([-2,-1,1,2])    
            trainX_3 =  img_array[new_z2 - 32: new_z2 + 32,
                                  new_y2 - 32 : new_y2 + 32,
                                  new_x2 - 32 : new_x2 + 32] 
            
            if trainX_3.shape == (64,64,64):
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_6_')+ '_3d_20_6_i.npy',trainX_3)
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_7_')+ '_3d_20_6_i.npy',np.fliplr(trainX_3))
                np.save(output_true + str(patient_id)+'_'+str(node_idx1)+'_'+str('_8_')+ '_3d_20_6_i.npy',np.flipud(trainX_3))

                        
                  

        for node_idx2, cur_row2 in mini_df_pred.iterrows():       
            node_x = cur_row2["coordX"]
            node_y = cur_row2["coordY"]
            node_z = cur_row2["coordZ"]
            diam = cur_row2["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            
            
            new_x = int(v_center[0])
            new_y = int(v_center[1])
            new_z = int(v_center[2])
            
            trainX_4 =   img_array[new_z - 32: new_z + 32,
                                 new_y - 32: new_y + 32,
                                 new_x - 32: new_x + 32] 
    
            if trainX_4.shape == (64,64,64):
                np.save(output_false + str(patient_id)+'_'+str(node_idx2)+'_'+str('_0_') + '_3d_20_6_i.npy',trainX_4)
    return

In [3]:
csv_path = PATH['annotations_train']
output_true = PATH['cls_train_cube_64_true']
output_false = PATH['cls_train_cube_64_false']
pred_csv_path = PATH['model_train_pred']
data_path = PATH['model_train_pred']
anno_csv_new = pd.read_csv(csv_path + "annotations_all.csv")
pred_csv_new = pd.read_csv(pred_csv_path + "anno_false_final.csv")

In [4]:
patients = [x for x in os.listdir(data_path) if 'pred.npy' in x] 

In [5]:
anno_csv_new["file"] = anno_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
anno_csv_new = anno_csv_new.dropna()
pred_csv_new["file"] = pred_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
pred_csv_new = pred_csv_new.dropna()

In [6]:
Parallel(n_jobs=-1)(delayed(create_cls_sample)(anno_csv_new,pred_csv_new,patient,data_path,output_true,output_false) for patient in tqdm(sorted(patients)))

100%|██████████| 800/800 [35:21<00:00,  2.41s/it]  


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,